In [22]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [23]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

### Retrieval function

In [24]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [25]:
def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context= []
    similiarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload['parent_asin'])
        retrieved_context.append(result.payload['description'])
        similiarity_scores.append(result.score)
        retrieved_context_ratings.append(result.payload['average_rating'])

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similiarity_scores": similiarity_scores,
        "retrieved_context_ratings": retrieved_context_ratings
    }



In [26]:
retrieved_context = retrieve_data("What kind of earphones do you offer?", qdrant_client, k=10)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `Amazon-items-collection-00` doesn\'t exist!"},"time":0.000365208}'

In [ ]:
retrieved_context

{'retrieved_context_ids': ['B0B3HYZ427',
  'B0C3ZQ7KHJ',
  'B0B2W7QJXT',
  'B0B7FFK66G',
  'B09TZPTR9N',
  'B0B5GPCS2V',
  'B0BHKMQ7PN',
  'B08FZJP2TL',
  'B096BND451',
  'B0BRQRS9BM'],
 'retrieved_context': ['Tronsmart Wireless Earbuds, Dual-Driver Earbuds Bluetooth 5.3 with 3 EQ Modes, in-Ear Bluetooth Headphones IPX5 Waterproof, Stereo Wireless Earphones 32Hrs Playtime for Music, Workout, etc. Black 🎶【Pure Sound Quality】-The wireless earbuds are equipped with a 10mm Dynamic Driver and a balanced Armature Driver, this dual driver technology could deliver high and low-frequency sound directly to your ear without interference. Its wide soundstage is detailed and spacious, the bass has a deep punch, mids are luscious, and treble sparkles. 🎼【3EQ Modes】-The 3EQ modes of these Bluetooth earbuds will upgrade your music experience. The Bass Booster enhances bass-heavy music, Pure makes voices stand out clearly, and the default EQ (Normal) delivers well-balanced sound. To switch to a differen

### Format retrieved context function

In [9]:
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context['retrieved_context_ids'], context['retrieved_context'], context['retrieved_context_ratings']):
        formatted_context += f"- ID: {id}, Rating: {rating}, Description: {chunk}\n"

    return formatted_context

In [11]:
proprocessed_context = process_context(retrieved_context)
print(proprocessed_context)

- ID: B0B3HYZ427, Rating: 4.1, Description: Tronsmart Wireless Earbuds, Dual-Driver Earbuds Bluetooth 5.3 with 3 EQ Modes, in-Ear Bluetooth Headphones IPX5 Waterproof, Stereo Wireless Earphones 32Hrs Playtime for Music, Workout, etc. Black 🎶【Pure Sound Quality】-The wireless earbuds are equipped with a 10mm Dynamic Driver and a balanced Armature Driver, this dual driver technology could deliver high and low-frequency sound directly to your ear without interference. Its wide soundstage is detailed and spacious, the bass has a deep punch, mids are luscious, and treble sparkles. 🎼【3EQ Modes】-The 3EQ modes of these Bluetooth earbuds will upgrade your music experience. The Bass Booster enhances bass-heavy music, Pure makes voices stand out clearly, and the default EQ (Normal) delivers well-balanced sound. To switch to a different mode, quickly press the left bud 3 times. 🦻【One Step Pairing】-The moment you open the charging case, the Bluetooth Headphones will automatically connect to your dev

### Create prompt function

In [12]:
def build_prompt(preprocessed_context, question):
    prompt = f"""
    You are a shopping assistent that can answer questions about the products in stock.
    
    You will be given a question and a list of context.
    
    Instructions:
    - You need to answer the questions based on the provided context only.
    - Never use word context and refer to it as a the available products.
    
    Context:
    {preprocessed_context}
    
    Question:
    {question}
    """

    return prompt

In [13]:
prompt = build_prompt(proprocessed_context, "What kind of earphones do you offer?")

In [15]:
print(prompt)


    You are a shopping assistent that can answer questions about the products in stock.

    You will be given a question and a list of context.

    Instructions:
    - You need to answer the questions based on the provided context only.
    - Never use word context and refer to it as a the available products.

    Context:
    - ID: B0B3HYZ427, Rating: 4.1, Description: Tronsmart Wireless Earbuds, Dual-Driver Earbuds Bluetooth 5.3 with 3 EQ Modes, in-Ear Bluetooth Headphones IPX5 Waterproof, Stereo Wireless Earphones 32Hrs Playtime for Music, Workout, etc. Black 🎶【Pure Sound Quality】-The wireless earbuds are equipped with a 10mm Dynamic Driver and a balanced Armature Driver, this dual driver technology could deliver high and low-frequency sound directly to your ear without interference. Its wide soundstage is detailed and spacious, the bass has a deep punch, mids are luscious, and treble sparkles. 🎼【3EQ Modes】-The 3EQ modes of these Bluetooth earbuds will upgrade your music experien

### Generate answer function

In [16]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


In [17]:
print(generate_answer(prompt))

We offer a variety of earphones, including:

- Tronsmart Wireless Earbuds: Bluetooth 5.3, dual-driver (10mm dynamic + balanced armature), 3 EQ modes, IPX5, ~32 hours total playtime with case, lightweight and comfortable.
- Other wireless earbuds: 90H playtime with a 2200mAh case, IPX7 sweatproof, over-ear with ear hooks, quick charge, and dual LED power display.
- Bone conduction open-ear headphones: IPX5 sweatproof, bone conduction tech, lightweight for sports.
- Various protective cases and accessories for earphones (e.g., protective cases, carabiners, etc.), though these are not earphones themselves.

If you’d like, I can narrow it down to the exact features you want (e.g., battery life, water resistance, form factor).


### Combine RAG pipeline

In [18]:
def rag_pipeline(question, k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer


In [21]:
print(rag_pipeline("What kind of earphones can I get with rating above 4.5?"))

You can get Bluetooth Headphones Wireless Earbuds with 90H Playtime and 2200mAh charging case (ID B0C3ZQ7KHJ) rated 4.8.
